In [1]:
!pip install sentence-transformers faiss-cpu pandas


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle


#  Load cleaned dataset

df = pd.read_csv("shl_cleaned.csv")
print("Rows:", len(df))


#  loading ther  embedding model
model_name = "BAAI/bge-small-en-v1.5"
model = SentenceTransformer(model_name)
print("Model loaded:", model_name)

# Create text representation
def build_text(row):
    return f"""
    Name: {row['Assessment Name']}
    Description: {row['Description']}
    Test Type: {row['Test Type']}
    Remote: {row['Remote Support (Yes/No)']}
    Adaptive: {row['Adaptive/IRT (Yes/No)']}
    """

df["full_text"] = df.apply(build_text, axis=1)

#  Generate embeddings

texts = df["full_text"].tolist()
embeddings = model.encode(texts, batch_size=32, convert_to_numpy=True, normalize_embeddings=True)

print("Embeddings shape:", embeddings.shape)


#  Build FAISS index

dimension = embeddings.shape[1]  # embedding size (usually 384)
index = faiss.IndexFlatIP(dimension)  # cosine similarity via inner product
index.add(embeddings)

print("FAISS index built with", index.ntotal, "vectors")


# Save index + dataframe + model name

faiss.write_index(index, "shl_faiss.index")

with open("shl_metadata.pkl", "wb") as f:
    pickle.dump({
        "df": df,
        "model_name": model_name
    }, f)

print("Saved: shl_faiss.index + shl_metadata.pkl")


Rows: 377
Model loaded: BAAI/bge-small-en-v1.5
Embeddings shape: (377, 384)
FAISS index built with 377 vectors
Saved: shl_faiss.index + shl_metadata.pkl


In [5]:
# test 
query = "Looking for a Java developer with good teamwork skills"
query_emb = model.encode([query], normalize_embeddings=True)

D, I = index.search(query_emb, k=10)

for idx in I[0]:
    print(df.iloc[idx]["Assessment Name"], "|", df.iloc[idx]["Link"])


Java Design Patterns (New) | https://www.shl.com/products/product-catalog/view/java-design-patterns-new/
Core Java (Entry Level) (New) | https://www.shl.com/products/product-catalog/view/core-java-entry-level-new/
Java Frameworks (New) | https://www.shl.com/products/product-catalog/view/java-frameworks-new/
Java 8 (New) | https://www.shl.com/products/product-catalog/view/java-8-new/
Java Web Services (New) | https://www.shl.com/products/product-catalog/view/java-web-services-new/
Job Control Language (New) | https://www.shl.com/products/product-catalog/view/job-control-language-new/
Core Java (Advanced Level) (New) | https://www.shl.com/products/product-catalog/view/core-java-advanced-level-new/
Java 2 Platform Enterprise Edition 1.4 Fundamental | https://www.shl.com/products/product-catalog/view/java-2-platform-enterprise-edition-1-4-fundamental/
JavaScript (New) | https://www.shl.com/products/product-catalog/view/javascript-new/
Java Platform Enterprise Edition 7 (Java EE 7) | https: